In [35]:
import pandas as pd
%run 5_Funciones.ipynb
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

En primer lugar, voy a configurar un WebDriver para Selenium con opciones específicas, incluyendo deshabilitar la automatización para evitar la detección de bots, configurar el navegador como "headless" (invisible), maximizar la ventana y abrir una ventana de incógnito, utilizando el controlador de Chrome ubicado en 'drive/chromedriver':

In [2]:
opciones = Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless = False
opciones.add_argument('--start-maximized')
opciones.add_argument('--incognito')

In [3]:
PATH = 'drive/chromedriver'
driver = webdriver.Chrome(options=opciones)

A continuación, voy a utilizar el WebDriver para abrir y cargar la página web que me interesa:

In [4]:
url = "https://www.ign.es/web/ane-datos-geograficos/-/datos-geograficos/datosPoblacion?tipoBusqueda=capitales"
driver.get(url)

In [60]:
'''
Utilizo Selenium para localizar y almacenar en la variable 
tabla el elemento HTML de la tabla específica en la página web.
'''
tabla = driver.find_element(By.XPATH, '//*[@id="p_p_id_IGNANEDatosGeograficos_WAR_IGNANEDatosGeograficosportlet_"]/div/div/div/div/table[2]')
tabla

<selenium.webdriver.remote.webelement.WebElement (session="af9d9e57b3b151db1f516436b93e218e", element="404974492293EB46074518B2F2600239_element_44")>

In [61]:
'''
Localizo y obtengo todas las filas (<tr>) dentro del 
cuerpo (<tbody>) de la tabla utilizando Selenium.
'''
cuerpo = tabla.find_element(By.TAG_NAME, 'tbody')

filas = cuerpo.find_elements(By.TAG_NAME, 'tr')

In [62]:
'''
Itero sobre las filas de una tabla, extrayendo el 
texto de las celdas (<td>) en cada fila y almacenando 
esos datos en una lista 'datos_lista'
'''
datos_filas = []
for fila in filas:
    celdas = fila.find_elements(By.TAG_NAME, 'td')
    datos_fila = [celda.text for celda in celdas]
    datos_filas.append(datos_fila)

Una vez realizada la extracción, creo un DataFrame de Pandas llamado data utilizando la información extraída de las filas de la tabla y asignando nombres de columna específicos:

In [63]:
columnas = ['Capital', 'Provincia', 'Población', 'Latitud', 'Longitud', 'XUTM ETRS89', 'YUTM ETRS89', 
            'Altura (m)', 'Huso']
data = pd.DataFrame(datos_filas, columns=columnas)

In [64]:
data.dtypes

Capital        object
Provincia      object
Población      object
Latitud        object
Longitud       object
XUTM ETRS89    object
YUTM ETRS89    object
Altura (m)     object
Huso           object
dtype: object

Acontinuación, elimino las columnas y filas (Ceuta y Melilla) que no me interesan para el análisis:

In [65]:
data= data.drop(columns=['XUTM ETRS89', 'YUTM ETRS89', 'Altura (m)', 'Capital', 'Huso'])

In [66]:
filas_eliminar= [0,13,30]
data= data.drop(filas_eliminar).reset_index(drop=True)
data.head()

,Provincia,Población,Latitud,Longitud
0,A Coruña,246.047,"43,37012643","-8,39114853"
1,Albacete,170.475,"38,99588053","-1,85574745"
2,Alicante / Alacant,334.418,"38,34548705","-0,4831832"
3,Almería,190.013,"36,83892362","-2,46413188"
4,Ávila,58.245,"40,65586958","-4,69771277"


Realizaré una corrección de nombres de provincias en la columna 'Provincia' del DataFrame 'data' mediante un diccionario de mapeo para que en todos los CSV estén estandarizados:

In [67]:
correccion_provincias={'Álava': 'Araba/alava',
                      'Santa Cruz de Tenerife':'Sta. cruz de tenerife',
                      'Málaga':'Malaga',
                      'Las Palmas': 'Las palmas de gran canaria',
                      'Valencia / València':'Valencia',
                      'La Rioja': 'La rioja',
                      'León':'Leon',
                      'Alicante / Alacant':'Alicante',
                      'Almería':'Almeria',
                      'Ávila': 'Avila',
                      'Illes Balears': 'Islas baleares',
                      'Cáceres': 'Caceres',
                      'Cádiz':'Cadiz',
                      'Castellón / Castelló': 'Castellon',
                      'Ciudad Real': 'Ciudad real',
                      'Córdoba': 'Cordoba',
                      'A Coruña': 'A coruna',
                      'Jaén': 'Jaen' }

data['Provincia']= data['Provincia'].replace(correccion_provincias)

Trato los tipos de datos:

In [68]:
data['Población'] = pd.to_numeric(data['Población'].str.replace('.', ''), errors='coerce').astype(int)
data['Latitud'] = pd.to_numeric(data['Latitud'].str.replace(',', '.'), errors='coerce')
data['Longitud'] = pd.to_numeric(data['Longitud'].str.replace(',', '.'), errors='coerce')

In [69]:
data.dtypes

Provincia     object
Población      int32
Latitud      float64
Longitud     float64
dtype: object

In [71]:
data.to_csv('../data/IGN.csv', index=False)